In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import talib as ta
import tensorflow as tf
from scipy.stats import norm
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model

In [2]:
# Definingclass to calculate technical indicators
class TechnicalIndicators:
    def __init__(self, data):
        self.data = data

    def add_momentum_indicators(self):
        self.data['RSI'] = ta.RSI(self.data['Close'], timeperiod=14)
        self.data['MACD'], self.data['MACD_signal'], self.data['MACD_hist'] = ta.MACD(self.data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        self.data['Stoch_k'], self.data['Stoch_d'] = ta.STOCH(self.data['High'], self.data['Low'], self.data['Close'],
                                                              fastk_period=14, slowk_period=3, slowd_period=3)

    def add_volume_indicators(self):
        self.data['OBV'] = ta.OBV(self.data['Close'], self.data['Volume'])

    def add_volatility_indicators(self):
        self.data['Upper_BB'], self.data['Middle_BB'], self.data['Lower_BB'] = ta.BBANDS(self.data['Close'], timeperiod=20)
        self.data['ATR_1'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=1)
        self.data['ATR_2'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=2)
        self.data['ATR_5'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)
        self.data['ATR_10'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=10)
        self.data['ATR_20'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=20)

    def add_trend_indicators(self):
        self.data['ADX'] = ta.ADX(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['+DI'] = ta.PLUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['-DI'] = ta.MINUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['CCI'] = ta.CCI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)

    def add_other_indicators(self):
        self.data['DLR'] = np.log(self.data['Close'] / self.data['Close'].shift(1))
        self.data['TWAP'] = self.data['Close'].expanding().mean()
        self.data['VWAP'] = (self.data['Volume'] * (self.data['High'] + self.data['Low']) / 2).cumsum() / self.data['Volume'].cumsum()

    def add_all_indicators(self):
        self.add_momentum_indicators()
        self.add_volume_indicators()
        self.add_volatility_indicators()
        self.add_trend_indicators()
        self.add_other_indicators()
        return self.data

In [3]:
# Loading the data and preprocessing the data
data = pd.read_csv('xnas-itch-20230703.tbbo.csv')

data['price'] = data['price'] / 1e9
data['bid_px_00'] = data['bid_px_00'] / 1e9
data['ask_px_00'] = data['ask_px_00'] / 1e9

data['Close'] = data['price']
data['Volume'] = data['size']
data['High'] = data[['bid_px_00', 'ask_px_00']].max(axis=1)
data['Low'] = data[['bid_px_00', 'ask_px_00']].min(axis=1)
data['Open'] = data['Close'].shift(1).fillna(data['Close'])

# Calculate pct_change (percentage change in price)
data['pct_change'] = data['price'].pct_change()
data['liquidity'] = data['bid_sz_00'] * data['bid_px_00'] + data['ask_sz_00'] * data['ask_px_00']

window_size = 60  # Example window size, adjust as needed
data['rolling_mean_vol'] = data['pct_change'].rolling(window=window_size).mean()
data['rolling_std_vol'] = data['pct_change'].rolling(window=window_size).std()
data['rolling_mean_liq'] = data['liquidity'].rolling(window=window_size).mean()
data['rolling_std_liq'] = data['liquidity'].rolling(window=window_size).std()

In [4]:
# Calculating all technical indicators
ti = TechnicalIndicators(data)
df_with_indicators = ti.add_all_indicators()


market_features_df = df_with_indicators.dropna().reset_index(drop=True)

print(market_features_df.head())

               ts_recv             ts_event  rtype  publisher_id  \
0  1688371262739229179  1688371262739065129      1             2   
1  1688371262739239338  1688371262739074592      1             2   
2  1688371270151139658  1688371270150974638      1             2   
3  1688371271298128899  1688371271297964469      1             2   
4  1688371271609555288  1688371271609390022      1             2   

   instrument_id action side  depth   price  size  ...     ATR_5    ATR_10  \
0             32      T    B      0  194.12    75  ...  0.028307  0.059306   
1             32      T    B      0  194.12     9  ...  0.024646  0.054375   
2             32      T    B      0  194.18     1  ...  0.033717  0.055938   
3             32      T    B      0  194.18     2  ...  0.038973  0.056344   
4             32      T    B      0  194.18     1  ...  0.043179  0.056710   

     ATR_20        ADX        +DI       -DI         CCI       DLR        TWAP  \
0  0.078831  93.017224   8.091754  0.3388

In [5]:
# Define the columns that will be used as input features
feature_columns = [
    'price', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'Stoch_k', 
    'Stoch_d', 'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 
    'ATR_2', 'ATR_5', 'ATR_10', 'ATR_20', 'ADX', '+DI', '-DI', 'CCI'
]

def preprocess_data_for_transformer(data, feature_columns, sequence_length=60):
    # Extract the features
    data_values = data[feature_columns].values
    
    # Convert to sequences
    X = []
    y = []
    for i in range(sequence_length, len(data_values)):
        X.append(data_values[i-sequence_length:i])
        y.append(data_values[i, 0])

    X = np.array(X)
    y = np.array(y)
    
    return X, y

# Preprocessing the ticker data
X, y = preprocess_data_for_transformer(market_features_df, feature_columns)


In [6]:
# Defining the Transformer model
def transformer_block(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = Dense(ff_dim, activation="relu")(res)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    return x + res

def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_blocks, dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs

    for _ in range(num_blocks):
        x = transformer_block(x, head_size, num_heads, ff_dim, dropout)

    x = Dense(20, activation="relu")(x)
    x = Dense(10, activation="relu")(x)
    outputs = Dense(1, activation="linear")(x[:, -1])  # Predict the next value in the sequence

    model = Model(inputs, outputs)
    return model

input_shape = (X.shape[1], X.shape[2])  # (sequence_length, num_features)
model = build_transformer_model(input_shape, head_size=64, num_heads=4, ff_dim=64, num_blocks=2, dropout=0.1)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss="mse")
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 60, 20)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 20)    │     21,268 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 60, 20)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 60, 20)    │         40 │ dropout_1[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 20)    │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 60, 64)    │      1,344 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 60, 64)    │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 60, 20)    │      1,300 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 20)    │         40 │ dense_1[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 20)    │          0 │ layer_normalizat… │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 20)    │     21,268 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 60, 20)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 20)    │         40 │ dropout_4[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 20)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 60, 64)    │      1,344 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 60, 64)    │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 60, 20)    │      1,300 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 20)    │         40 │ dense_3[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 60, 20)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 48,625 (189.94 KB)

 Trainable params: 48,625 (189.94 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Splitting the data into training and validation sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

# Defining callbacks for fine-tuning
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6)

# Fine-tuning the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluating the fine-tuned model
val_loss = model.evaluate(X_val, y_val)
print(f"Validation Loss after fine-tuning: {val_loss}")


Epoch 1/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 44s 114ms/step - loss: 17038991360.0000 - val_loss: 5642279424.0000 - learning_rate: 5.0000e-05
Epoch 2/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 42s 112ms/step - loss: 3327272704.0000 - val_loss: 658518208.0000 - learning_rate: 5.0000e-05
Epoch 3/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 43s 117ms/step - loss: 339494592.0000 - val_loss: 30693448.0000 - learning_rate: 5.0000e-05
Epoch 4/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 43s 117ms/step - loss: 13711503.0000 - val_loss: 473513.9688 - learning_rate: 5.0000e-05
Epoch 5/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 43s 115ms/step - loss: 188132.3906 - val_loss: 1980.0991 - learning_rate: 5.0000e-05
Epoch 6/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 42s 114ms/step - loss: 1248.1782 - val_loss: 383.2761 - learning_rate: 5.0000e-05
Epoch 7/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 43s 115ms/step - loss: 502.6607 - val_loss: 399.9815 - learning_rate: 5.0000e-05
Epoch 8/100
370/370 ━━━━━━━━━━━━━━━━━━━━ 43s 116ms/step - loss: 495.4775 - val_loss: 405.1356 

In [ ]:
def simulate_trades(predictions, data, threshold=0.02):
    blotter = []
    balance = 10_000_000  # $10 million initial balance
    shares_held = 0
    portfolio = {'cash': balance, 'holdings': {ticker: 0 for ticker in data['symbol'].unique()}}
    cumulative_reward = 0

    for i in range(1, len(predictions)):
        row = data.iloc[i]
        predicted_price = predictions[i]
        actual_price = row['price']
        current_rsi = row['RSI']
        current_vol = row['pct_change']
        current_liq = row['liquidity']
        mean_vol = row['rolling_mean_vol']
        std_vol = row['rolling_std_vol']
        mean_liq = row['rolling_mean_liq']
        std_liq = row['rolling_std_liq']
        transaction_time = row['ts_in_delta']
        action = "Hold" 

        if current_rsi < 30: 
            trade_direction = 'BUY'
            trade_price = get_trade_price(actual_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction)
            trade_size = (portfolio['cash'] * np.random.uniform(0.001, 0.005)) / trade_price
            if portfolio['cash'] >= trade_size * trade_price:
                portfolio['cash'] -= trade_size * trade_price
                portfolio['holdings'][row['symbol']] += trade_size
                action = "Buy"
            else:
                action = "Cancelled"
        elif current_rsi > 70: 
            trade_direction = 'SELL'
            if portfolio['holdings'][row['symbol']] > 0:
                trade_size = min(portfolio['holdings'][row['symbol']], portfolio['cash'] * np.random.uniform(0.001, 0.005) / actual_price)
                trade_price = get_trade_price(actual_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction)
                portfolio['cash'] += trade_size * trade_price
                portfolio['holdings'][row['symbol']] -= trade_size
                action = "Sell"
            else:
                action = "Cancelled"

        if action != "Hold" and action != "Cancelled":
            expected_price = row['ask_px_00']
            transaction_cost = _calculate_transaction_cost(row['Volume'], 0.3, data['Volume'].mean())
            slippage = expected_price - actual_price
            time_penalty = 1000 * transaction_time / 1e9
            reward = -(slippage + time_penalty + transaction_cost)

            cumulative_reward += reward
            blotter.append({
                "Action": action,
                "Price": actual_price,
                "Shares Held": portfolio['holdings'][row['symbol']],
                "Balance": portfolio['cash'],
                "Portfolio Value": portfolio['cash'] + portfolio['holdings'][row['symbol']] * actual_price,
                "Transaction Cost": transaction_cost,
                "Reward": reward,
                "Slippage": slippage,
                "Time Penalty": time_penalty
            })

    # Returning the blotter and the final portfolio value
    final_portfolio_value = portfolio['cash'] + portfolio['holdings'][row['symbol']] * actual_price
    return blotter, final_portfolio_value, cumulative_reward

def _calculate_transaction_cost(volume, volatility, daily_volume):
    return volatility * np.sqrt(volume / daily_volume)

def get_trade_price(base_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction):
    vol_percentile = get_percentile(current_vol, mean_vol, std_vol)
    liq_percentile = get_percentile(current_liq, mean_liq, std_liq)

    if vol_percentile >= 0.9 and liq_percentile < 0.1:
        price_adjustment_percent = np.random.uniform(-0.25, -0.15)
    elif vol_percentile <= 0.1 and liq_percentile < 0.1:
        price_adjustment_percent = np.random.uniform(-0.10, -0.05)
    elif vol_percentile >= 0.9 and liq_percentile >= 0.9:
        price_adjustment_percent = np.random.uniform(-0.05, +0.10)
    else:
        price_adjustment_percent = np.random.uniform(-0.05, +0.05)

    if trade_direction == 'BUY':
        adjusted_price = base_price * (1 - price_adjustment_percent)
    else:
        adjusted_price = base_price * (1 + price_adjustment_percent)
    
    return adjusted_price

def get_percentile(current_value, mean, std):
    if std > 0:
        z_score = (current_value - mean) / std
        percentile = norm.cdf(z_score)
    else:
        percentile = 0.5
    return percentile

In [ ]:

# Simulate trades based
y_pred = model.predict(X_val)

blotter, final_portfolio_value, cumulative_reward = simulate_trades(y_pred, market_features_df)


blotter_df = pd.DataFrame(blotter)

# Evaluate the performance
total_profit = final_portfolio_value - 10_000_000  # Initial cash
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Profit: ${total_profit:.2f}")
print(f"Cumulative Reward: {cumulative_reward:.2f}")

blotter_df.to_csv('transformer_model_blotter.csv', index=False)
